In [1]:
import numpy as np
import random

In [10]:
A = [4, 6, 8, 10, 12]
k = [33, 16, 11, 8, 6]
sigs = 25
for i in range(len(A)):
    print(f"When A = {A[i]}, and k = {k[i]}, eps = {A[i] * sigs * k[i]}")

When A = 4, and k = 33, eps = 3300
When A = 6, and k = 16, eps = 2400
When A = 8, and k = 11, eps = 2200
When A = 10, and k = 8, eps = 2000
When A = 12, and k = 6, eps = 1800


In [166]:
# Define the alphabet
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .'

# Convert text to binary sequence
def text_to_binary(text):
    binary_seq = ''.join(format(alphabet.index(c), '06b') for c in text)
    return np.array([int(b) for b in binary_seq])

codewords = []
for i in range(32):
    p = np.zeros(32)
    p[i] = 70
    codewords.append(p)
    q = np.zeros(32)
    q[i] = -70
    codewords.append(q)

CODEBOOK = {}
for i in range(64):
    CODEBOOK[alphabet[i]] = codewords[i]

# Convert binary sequence to codeword with n samples
def binary_to_codeword(binary_seq, n):
    codeword = np.zeros(len(binary_seq)*n)
    for i in range(len(binary_seq)):
        codeword[i*n:(i+1)*n] = (-1)**(int(binary_seq[i]))  # Map 0 to 1 and 1 to -1
    return codeword

In [167]:
for i in CODEBOOK.values():
    print(i)

[70.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[-70.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
[ 0. 70.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[  0. -70.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
[ 0.  0. 70.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[  0.   0. -70.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
[ 0.  0.  0. 70.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [168]:
print(text_to_binary('b'))
print(len(binary_to_codeword(text_to_binary('H'), 32)))

[0 0 0 0 0 1]
192


In [169]:
print(binary_to_codeword('000000', 32))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [172]:
# Convert binary sequence back to text
def binary_to_text(binary_seq):
    binary_str = ''.join(map(str, binary_seq))
    text = ''.join(alphabet[int(binary_str[i:i+6], 2)] for i in range(0, len(binary_str), 6))
    return text

# Simulate the channel
def channel(x):
    n = x.size
    B = random.choice([0, 1])
    sigma_sq = 25
    Z = np.random.normal(0, np.sqrt(sigma_sq), 2 * n)
    X = np.zeros(2 * n)
    if B == 1:
        X[:n] = x
    else:
        X[n:] = x
    Y = X + Z
    Y = np.reshape(Y, (-1))
    return Y

# Save codebook to a file
def save_codebook_to_file(codebook, file_path):
    np.save(file_path, codebook)

# Load codebook from a file
def load_codebook_from_file(file_path):
    return np.load(file_path)



# Transmitter
message = "bbaaaaaaaaaaaaa.aaaa123aaaaaaaaaaaaaaaba"
print("Original Message:", message)

#print("Codeword:", binary_seq)
    #np.savetxt('input_signal.txt', codeword)

    # Channel
    #transmitted_signal = np.loadtxt('input_signal.txt')
transmitted_signal = np.zeros(40*32)
for i in range(40):
    transmitted_signal[i*32:(i+1)*32] = CODEBOOK[message[i]]

received_signal = channel(transmitted_signal)
    #np.savetxt('received_signal.txt', received_signal)
print("Received Signal:", received_signal)

    


Original Message: bbaaaaaaaaaaaaa.aaaa123aaaaaaaaaaaaaaaba
Received Signal: [-75.7113657   -1.97266155  -4.30032657 ...   3.57296985   2.32475361
  -4.67113677]


In [173]:
# Receiver
    #received_signal = np.loadtxt('received_signal.txt')
message_code1 = []
message_code2 = []
for i in range(40):
    message_code1.append(received_signal[i*32:(i+1)*32])
    message_code2.append(received_signal[len(received_signal)//2+i*32:len(received_signal)//2+(i+1)*32])
message_bin = []
for i in range(40):
    char_bin = ''
    y1 = message_code1[i]
    y2 = message_code2[i]
    i1 = np.argmin([np.linalg.norm(y1 - c) for c in CODEBOOK.values()])
    char1 = alphabet[i1]
    i2 = np.argmin([np.linalg.norm(y2 - c) for c in CODEBOOK.values()])
    char2 = alphabet[i2]
    d1 = i1 ** 2 / 25 + np.linalg.norm(y2) ** 2 / 25
    d2 = i2 ** 2 / 25 + np.linalg.norm(y1) ** 2 / 25
    if d1 < d2:
        char_bin = char1
    else:
        char_bin = char2  
    message_bin.append(char_bin)

msg = ''
for i in range(40):
    msg += (message_bin[i])

print(msg)


bbaaaaaaaaaaaaa.aaaa123aaaaaaaaaaaaaaaba


In [174]:
print(CODEBOOK['0'])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 70.  0.  0.  0.  0.  0.]
